In [64]:
import  tensorflow as tf 
import  numpy as np
import os




Lendo os dados

In [97]:
text = open("poemas.txt", "rb").read().decode(encoding='utf-8')
print(f'Tamanho do corpus: {len(text)} caracteres')

Tamanho do corpus: 113840 caracteres


In [66]:
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

86 caracteres únicos


Vetorizando o texto

In [67]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [68]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '"' :   2,
  '(' :   3,
  ')' :   4,
  ',' :   5,
  '-' :   6,
  '.' :   7,
  '0' :   8,
  '1' :   9,
  '2' :  10,
  '3' :  11,
  '4' :  12,
  '5' :  13,
  '6' :  14,
  '7' :  15,
  '8' :  16,
  '9' :  17,
  ':' :  18,
  ';' :  19,
  ...
}


In [69]:
print(f'{text[:13]} ---- mapeado para ---- > {repr(text[:13]), text_as_int[:13]}')


Imagine uma n ---- mapeado para ---- > ("'Imagine uma n'", array([29, 53, 42, 48, 50, 54, 46,  0, 61, 53, 42,  0, 54]))


A tarefa de previsão

In [70]:
seq_length = 100
examples_per_epoch = len(text)//seq_length

#cria os exemplos de treinamento e os targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(7):
    print(idx2char[i.numpy()])

I
m
a
g
i
n
e


In [71]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(7):
    print(repr(''.join(idx2char[item.numpy()])))

'Imagine uma nova história para sua vida e acredite nela.Tudo o que um sonho precisa para ser realizad'
'o é alguém que acredite que ele possa ser realizado.Pessimismo leva à fraqueza, otimismo ao poder.Não'
' deixe que as pessoas te façam desistir daquilo que você mais quer na vida. Acredite. Lute. Conquiste'
'. E acima de tudo, seja feliz!Lute com determinação, abrace a vida com paixão, perca com classe e ven'
'ça com ousadia, porque o mundo pertence a quem se atreve e a vida é muito para ser insignificante.O p'
'essimista vê dificuldade em cada oportunidade; o otimista vê oportunidade em cada dificuldade.Diga o '
'que você pensa com esperança.Pense no que você faz com fé.Faça o que você deve fazer com amor!A esper'


In [72]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [73]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Imagine uma nova história para sua vida e acredite nela.Tudo o que um sonho precisa para ser realiza'
Target data: 'magine uma nova história para sua vida e acredite nela.Tudo o que um sonho precisa para ser realizad'


In [74]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 29 ('I')
  expected output: 53 ('m')
Step    1
  input: 53 ('m')
  expected output: 42 ('a')
Step    2
  input: 42 ('a')
  expected output: 48 ('g')
Step    3
  input: 48 ('g')
  expected output: 50 ('i')
Step    4
  input: 50 ('i')
  expected output: 54 ('n')


In [75]:
#cria os batches
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Construindo o modelo

    Usamos tf.keras.Sequential para definir o modelo. Para este exemplo simples, três camadas são usadas para definir nosso modelo:

    tf.keras.layers.Embedding : A camada de entrada. Uma tabela de pesquisa treinável que mapeará os números de cada caractere para um vetor com dimensões embedding_dim ;

    tf.keras.layers.GRU : Um tipo de RNN com units=rnn_units tamanho units=rnn_units (também poderíamos usar uma camada LSTM aqui.)

    tf.keras.layers.Dense : A camada de saída, com saídas vocab_size .


In [76]:
# Tamanho do vocabulário em caracteres
vocab_size = len(vocab)

# Dimensão dos embeddings 
embedding_dim = 256

# Número de unidades da RNN 
rnn_units = 1024

In [77]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [78]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


In [79]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [80]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 86) # (batch_size, sequence_length, vocab_size)


In [81]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (64, 100, 256)         │        22,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (64, 100, 1024)        │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (64, 100, 86)          │        88,150 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,048,470 (15.44 MB)

 Trainable params: 4,048,470 (15.44 MB)

 Non-trainable params: 0 (0.00 B)

In [82]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [83]:
sampled_indices


array([76, 11, 82, 21,  0, 31, 41, 18, 79, 14, 25, 18, 29, 59, 62, 27, 55,
       56, 85, 34, 79, 34, 73, 32, 65, 42, 10, 56, 62, 41, 28, 48, 68, 30,
       77, 20, 30, 74, 29, 36, 67, 18, 73, 49,  0, 48, 81,  2, 45, 25, 49,
       76, 40, 23,  2, 32, 12, 63, 53,  1, 17, 46, 20, 48, 42, 13, 51, 44,
       78, 81, 78, 23, 51, 32, 10, 62, 21, 14, 31,  8, 25, 16, 63, 28, 82,
       63, 13, 35, 83, 32, 33, 83, 12, 33, 45, 50, 45, 11, 56,  5],
      dtype=int64)

decodificar para  ver o textoprecisto por essemodelo não treinado

In [84]:
print("Input: \n", repr("".join(idx2char[input_example_batch[1]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ho.Eu sou mesmo quem eu sou!QUEM SOU EUSou uma pessoa feliz,Amo muito a vidaE dela sou aprendiz;Tenh'

Next Char Predictions: 
 'ó3—A LV:ú6E:IsvGop…OúOéMÀa2pvVHgàJô?JêIQÉ:éh g–"dEhóUC"M4xm!9e?ga5jcõ–õCjM2vA6L0E8xH—x5P“MN“4Ndid3p,'


treinando o modelo

In [85]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 86)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4511433


In [86]:
model.compile(optimizer='adam', loss=loss)


In [87]:
# Diretório em que os checkpoints serão salvos
checkpoint_dir = './training_checkpoints'

# Nome dos arquivos de checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [104]:
EPOCHS = 100


In [105]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - loss: 0.1689
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - loss: 0.1702
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 0.1650
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1590
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1583
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1501
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1497
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1462
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1435
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1394
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1375
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1371
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1357
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1324
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 0.1278
Epoc

gerar texto

In [106]:
tf.train.latest_checkpoint(checkpoint_dir)


In [107]:
checkpoint_file = "/c:/Users/nockk/OneDrive/Área de Trabalho/gerar frases/checkpoint_file.weights.h5"  # Replace with the actual file path
if checkpoint_file is not None and os.path.exists(checkpoint_file):
    if checkpoint_file is not None:
        model.load_weights(checkpoint_file)


In [108]:
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (64, 100, 256)         │        22,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (64, 100, 1024)        │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (64, 100, 86)          │        88,150 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,145,412 (46.33 MB)

 Trainable params: 4,048,470 (15.44 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,096,942 (30.89 MB)

In [109]:
def generate_text(model, start_string):
    
    # Número de caracteres a serem gerados
    num_generate = 500

    # Vetorização
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Lista vazia para guardar os resultados.
    text_generated = []

    temperature = 1.0

    for i in range(num_generate):
        predictions = model(input_eval)
        
        # remove a dimensão do batch
        predictions = predictions[0]

        # utiliza uma distribuição categórica para prever o caractere que o modelo retorna
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Passa o caractere previsto como próximo input para o modelo junto 
        # com o estado anterior da célula 
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [123]:
generated_text = generate_text(model, start_string=u" ")
print(generated_text)

# Salvar o texto gerado em um arquivo com quebras de linha
with open("generated_text.txt", "a", encoding='utf-8') as file:
    file.write(generated_text + '\n''\n')

 cada dificuldade.Diga o que visto.Eu sou do jeito que estou!Não sou também o que eu tenho.Eu sou mesmo quem eu sou!QUEM SOU EUSou uma pessoa feliz,Amo muito a vidaE dela sou aprendiz;Tenho várias paixões,Mas, como qualquer um,Possuo imperfeições;Se os caminhos desta vida é muito para ser insignificante.O pessimista vê dificuldade em cada oportunidade; o otimista vê oportunidade em cada dificuldade.Diga o que você pensa com esperança.Pense no que você faz com fé.Faça o que você deve fazer com amo


In [120]:
model.save('modelo.keras')
